## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

In [1]:
# Criando dados fictícios

import numpy as np

shape = (160, 64, 750)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data.shape

(160, 64, 750)

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [2]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

75918.03219546896

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [3]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    psd = np.abs(fft_result) ** 2
    frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
    target_amplitudes_trial = []
    for target_frequency in target_frequencies:
        # encontrando o índice da frequência alvo no espectro de frequência
        index = np.argmin(np.abs(frequencies - target_frequency))
        # amplitude na frequência alvo
        amplitude = np.sqrt(psd[index])
        target_amplitudes_trial.append(amplitude)
    target_amplitudes.append(target_amplitudes_trial)
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [4]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)


[[25.36694142 26.49515836 26.49515836 ... 16.93716792 25.21044517
  25.21044517]
 [20.5456129  24.81958636 24.81958636 ... 23.09128737 18.52360027
  18.52360027]
 [21.35710957 25.38310239 25.38310239 ... 24.1524762  24.01523001
  24.01523001]
 ...
 [21.71395385 21.41496243 21.41496243 ... 21.57546805 21.57538292
  21.57538292]
 [22.34468889 21.57996449 21.57996449 ... 23.71130576 22.08802106
  22.08802106]
 [26.21745223 22.97685385 22.97685385 ... 24.04684251 23.20559755
  23.20559755]]
(64, 40)
[[-32.61943525 -31.49121831 -31.49121831 ... -41.04920875 -32.7759315
  -32.7759315 ]
 [-37.44076377 -33.16679031 -33.16679031 ... -34.8950893  -39.4627764
  -39.4627764 ]
 [-36.6292671  -32.60327428 -32.60327428 ... -33.83390047 -33.97114666
  -33.97114666]
 ...
 [-36.27242282 -36.57141424 -36.57141424 ... -36.41090862 -36.41099375
  -36.41099375]
 [-35.64168778 -36.40641218 -36.40641218 ... -34.27507091 -35.89835561
  -35.89835561]
 [-31.76892444 -35.00952282 -35.00952282 ... -33.93953416 -34

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.